In [8]:
# Install dependencies
!pip install -q transformers datasets evaluate accelerate sentencepiece rouge_score


In [3]:
import pandas as pd
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import torch

# =====================
# 1. Load CSV datasets
# =====================
# Replace with your actual CSV paths
empathy_path = "/content/empathy_data_clean.csv"
counsel_path = "/content/mental_health_train.csv"

df_empathy = pd.read_csv(empathy_path, sep=",")  # empathy dataset might be comma-separated
df_counsel = pd.read_csv(counsel_path)

# Standardize columns: input -> "input_text", output -> "target_text"
df_empathy_proc = pd.DataFrame({
    "input_text": "As a caring listener, show empathy for: " + df_empathy["utterance"].astype(str),
    "target_text": df_empathy["prompt"].astype(str)
})

df_counsel_proc = pd.DataFrame({
    "input_text": "As a mental health counselor, provide therapeutic response to: " + df_counsel["Context"].astype(str),
    "target_text": df_counsel["Response"].astype(str)
})

# Convert to Hugging Face Dataset
ds_empathy = Dataset.from_pandas(df_empathy_proc)
ds_counsel = Dataset.from_pandas(df_counsel_proc)
dataset = concatenate_datasets([ds_empathy, ds_counsel]).train_test_split(test_size=0.2)

In [7]:
# =====================
# 2. Load tokenizer/model
# =====================
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# =====================
# 3. Tokenization
# =====================
max_input_len = 384
max_target_len = 192

def preprocess(batch):
    inputs = tokenizer(batch["input_text"], max_length=max_input_len, truncation=True, padding="max_length")
    targets = tokenizer(batch["target_text"], max_length=max_target_len, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)

# =====================
# 4. Data collator
# =====================
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# =====================
# 5. Metrics (ROUGE)
# =====================
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.argmax(-1) if isinstance(preds, tuple) else preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Only keep the standard metrics
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
        "rougeLsum": result["rougeLsum"],
    }

# =====================
# 6. Training arguments
# =====================
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_mentalhealth",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),   # ✅ mixed precision if GPU supports
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none"
)

# =====================
# 7. Trainer
# =====================
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# =====================
# 8. Train with Gradient Scaling (fp16 inside Trainer handles scaling)
# =====================
trainer.train()


Map:   0%|          | 0/64003 [00:00<?, ? examples/s]

Map:   0%|          | 0/16001 [00:00<?, ? examples/s]

/tmp/ipython-input-3491328471.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# =====================
# 9. Evaluate & Save
# =====================
metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)

trainer.save_model("./flan_t5_mentalhealth_final")
tokenizer.save_pretrained("./flan_t5_mentalhealth_final")